# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Loading Data

In [3]:
data = pd.read_csv("Data Faskes BPJS 2019.csv")
data.head()

,NoLink,Provinsi,KotaKab,Link,TipeFaskes,No,KodeFaskes,NamaFaskes,LatLongFaskes,AlamatFaskes,TelpFaskes
0,0,Nanggroe Aceh Darussalam,Kode Faskes dan Alamat Rumah Sakit BPJS di Kot...,https://lovia.life/id/health/bpjs/area/kota-la...,Rumah Sakit,1,0001R001,RSU Cut Nyak Dhien\r\n \r\n...,"http://maps.google.co.id/?q=4.488058,97.947963",Jl. Tm Bahrum No. 1 Langsa,\r\n 0641-0621039\r\n
1,0,Nanggroe Aceh Darussalam,Kode Faskes dan Alamat Rumah Sakit BPJS di Kot...,https://lovia.life/id/health/bpjs/area/kota-la...,Rumah Sakit,2,0001R004,RSU Cut Meutia Langsa\r\n \...,"http://maps.google.co.id/?q=4.488088,97.947781",Jl.Garuda Kebun Baru Langsa,\r\n 0641- 4840076\r\n
2,0,Nanggroe Aceh Darussalam,Kode Faskes dan Alamat Rumah Sakit BPJS di Kot...,https://lovia.life/id/health/bpjs/area/kota-la...,Rumah Sakit,3,0105R001,RSUD Langsa\r\n \r\n ...,"http://maps.google.co.id/?q=4.472208,97.975533",Jln.A.Yani No. 1 Langsa,\r\n 0641-22051\r\n
3,0,Nanggroe Aceh Darussalam,Kode Faskes dan Alamat Rumah Sakit BPJS di Kot...,https://lovia.life/id/health/bpjs/area/kota-la...,Rumah Sakit,4,0105R013,RSU Ummi \r\n \r\n ...,"http://maps.google.co.id/?q=4.470376,97.991915",Jln. Prof. A. Madjid Ibrahim,\r\n 0641-22886\r\n
4,0,Nanggroe Aceh Darussalam,Kode Faskes dan Alamat Rumah Sakit BPJS di Kot...,https://lovia.life/id/health/bpjs/area/kota-la...,Puskesmas,1,00010001,Langsa Lama\r\n \r\n ...,"http://maps.google.co.id/?q=4.478172,97.949988",Gampong Meurandeh Dayah,\r\n 0641-064121218\r\n


In [20]:
data.shape

(29157, 10)

# Data Cleansing

In [5]:
data.drop(['Link', 'NoLink'], axis=1, inplace=True)

In [6]:
# Fungsi ini untuk menghilangkan spasi berlebih dari sebuah string
def remSpace(row):
    # Membagi string berdasarkan spasi dan menggabungkannya kembali dengan satu spasi
    str = " ".join(row.split())
    return str

In [7]:
data['TelpFaskes'] = data['TelpFaskes'].apply(remSpace)
data['NamaFaskes'] = data['NamaFaskes'].apply(remSpace)
data['KotaKab'] = data['KotaKab'].apply(remSpace)

In [8]:
# Fungsi ini untuk mengambil nama kota dari teks yang berisi informasi alamat
def namaKota(row):
    # Menghapus bagian teks yang berisi "Kode Faskes dan Alamat Rumah Sakit BPJS di "
    str = row.replace('Kode Faskes dan Alamat Rumah Sakit BPJS di ','')
    return str

In [9]:
data['KotaKab'] = data['KotaKab'].apply(namaKota)

In [10]:
data.head()

,Provinsi,KotaKab,TipeFaskes,No,KodeFaskes,NamaFaskes,LatLongFaskes,AlamatFaskes,TelpFaskes
0,Nanggroe Aceh Darussalam,Kota Langsa,Rumah Sakit,1,0001R001,RSU Cut Nyak Dhien,"http://maps.google.co.id/?q=4.488058,97.947963",Jl. Tm Bahrum No. 1 Langsa,0641-0621039
1,Nanggroe Aceh Darussalam,Kota Langsa,Rumah Sakit,2,0001R004,RSU Cut Meutia Langsa,"http://maps.google.co.id/?q=4.488088,97.947781",Jl.Garuda Kebun Baru Langsa,0641- 4840076
2,Nanggroe Aceh Darussalam,Kota Langsa,Rumah Sakit,3,0105R001,RSUD Langsa,"http://maps.google.co.id/?q=4.472208,97.975533",Jln.A.Yani No. 1 Langsa,0641-22051
3,Nanggroe Aceh Darussalam,Kota Langsa,Rumah Sakit,4,0105R013,RSU Ummi,"http://maps.google.co.id/?q=4.470376,97.991915",Jln. Prof. A. Madjid Ibrahim,0641-22886
4,Nanggroe Aceh Darussalam,Kota Langsa,Puskesmas,1,00010001,Langsa Lama,"http://maps.google.co.id/?q=4.478172,97.949988",Gampong Meurandeh Dayah,0641-064121218


In [11]:
def searchLatLong(row):
    str = re.search('(-?([0-9]{1}|[0-9]0|[1-8]{1,2}).[0-9]{1,6},(-?(1[0-8]{1,2}|9[1-9]{1}).[1-9]{1,6}))', row)
    if str:
        return str.group()
    return np.NaN

In [12]:
data['LatLongFaskes'] = data['LatLongFaskes'].apply(searchLatLong)

In [13]:
lat = []
lon = []

for row in data['LatLongFaskes']:
    try:
        latitude = float(row.split(',')[0])
        longitude = float(row.split(',')[1])
        if (-90.0 <= latitude <= 90.0):
            lat.append(latitude)
        else:
            lat.append(np.NaN)

        if (-180 <= longitude <= 180):
            lon.append(longitude)
        else:
            lon.append(np.NaN)
    except:
        lat.append(np.NaN)
        lon.append(np.NaN)

data['Latitude'] = lat
data['Longitude'] = lon

In [14]:
data.drop(['LatLongFaskes'], axis=1, inplace=True)
data.head()

,Provinsi,KotaKab,TipeFaskes,No,KodeFaskes,NamaFaskes,AlamatFaskes,TelpFaskes,Latitude,Longitude
0,Nanggroe Aceh Darussalam,Kota Langsa,Rumah Sakit,1,0001R001,RSU Cut Nyak Dhien,Jl. Tm Bahrum No. 1 Langsa,0641-0621039,4.488058,97.947963
1,Nanggroe Aceh Darussalam,Kota Langsa,Rumah Sakit,2,0001R004,RSU Cut Meutia Langsa,Jl.Garuda Kebun Baru Langsa,0641- 4840076,4.488088,97.947781
2,Nanggroe Aceh Darussalam,Kota Langsa,Rumah Sakit,3,0105R001,RSUD Langsa,Jln.A.Yani No. 1 Langsa,0641-22051,4.472208,97.975533
3,Nanggroe Aceh Darussalam,Kota Langsa,Rumah Sakit,4,0105R013,RSU Ummi,Jln. Prof. A. Madjid Ibrahim,0641-22886,4.470376,97.991915
4,Nanggroe Aceh Darussalam,Kota Langsa,Puskesmas,1,00010001,Langsa Lama,Gampong Meurandeh Dayah,0641-064121218,4.478172,97.949988


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29157 entries, 0 to 29156
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Provinsi      29157 non-null  object 
 1   KotaKab       29157 non-null  object 
 2   TipeFaskes    29157 non-null  object 
 3   No            29157 non-null  object 
 4   KodeFaskes    29157 non-null  object 
 5   NamaFaskes    29157 non-null  object 
 6   AlamatFaskes  29157 non-null  object 
 7   TelpFaskes    29157 non-null  object 
 8   Latitude      16517 non-null  float64
 9   Longitude     19729 non-null  float64
dtypes: float64(2), object(8)
memory usage: 2.2+ MB


In [16]:
# Mengubah semua data menjadi string dan mengganti '-' dengan NaN
data = data.astype(str)
data = data.applymap(lambda x: re.sub(r'^-$', str(np.NaN), x))

In [17]:
data.to_excel('Data Faskes BPJS 2019-clean_data.xlsx', index=False)

In [18]:
data.to_csv('Data Faskes BPJS 2019-clean_data.csv', index=False)

In [19]:
data.columns

Index(['Provinsi', 'KotaKab', 'TipeFaskes', 'No', 'KodeFaskes', 'NamaFaskes',
       'AlamatFaskes', 'TelpFaskes', 'Latitude', 'Longitude'],
      dtype='object')